In [1]:
import os
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset

from sklearn.model_selection import KFold, StratifiedKFold

from torchvision import transforms, models
from torchvision.transforms import Resize, ToTensor, Normalize

In [2]:
# dataset path
train_dir = '/opt/ml/input/data/train'
test_dir = '/opt/ml/input/data/eval'

# csv files
train_csv = pd.read_csv(os.path.join(train_dir, 'train.csv'))
test_csv = pd.read_csv(os.path.join(test_dir, 'info.csv'))

# image directories
train_image_dir = os.path.join(train_dir, 'images')
test_image_dir = os.path.join(test_dir, 'images')

train_image_paths = [os.path.join(train_image_dir, img_id) for img_id in train_csv.path]
test_image_paths = [os.path.join(test_image_dir, img_id) for img_id in test_csv.ImageID]

In [3]:
# Define train datset & test datset class
class TrainDataset(Dataset):
    def __init__(self, img_paths, csv, transform):
        self.img_paths = img_paths
        self.csv = csv
        self.transform = transform

    def __getitem__(self, index):
        # get persion id and mask info
        pid, mask_id = divmod(index, 7)
        
        # get image
        mask_images = [d for d in os.listdir(self.img_paths[pid]) if not d.startswith('._')]
        image = Image.open(os.path.join(self.img_paths[pid], mask_images[mask_id]))

        if self.transform:
            image = self.transform(image)
            
        #get label info
        gender = 0 if self.csv.gender[pid] == 'male' else 1
        age = 0 if self.csv.age[pid] < 30 else 1 if 30 <= self.csv.age[pid] < 60 else 2
        mask = 1 if 'incorrect' in mask_images[mask_id] else 2 if 'normal' in mask_images[mask_id] else 0
        
        label = 6 * mask + 3 * gender + age
        
        return image, label
        

    def __len__(self):
        return len(self.img_paths) * 7
    
    
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [4]:
# Create model
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(2048, 18)

In [5]:
def collate_fn(batch):
    return tuple(zip(*batch))

transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    transforms.RandomHorizontalFlip(p=0.5),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])

train_dataset = TrainDataset(train_image_paths, train_csv, transform)
test_dataset = TestDataset(test_image_paths, transform)

train_data_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=4
)

test_data_loader = DataLoader(
    test_dataset,
    batch_size=8,
    shuffle=False,
    num_workers=4
)

In [6]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params, lr=0.00005, weight_decay=0.01)
# lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
lr_scheduler = None

criterion = nn.CrossEntropyLoss()

num_epochs = 10

kfold = KFold(n_splits=5)

for fold, (train_index, valid_index) in enumerate(kfold.split(train_dataset), 1):    
    train_sub = Subset(train_dataset, train_index)
    valid_sub = Subset(train_dataset, valid_index)
    
    train_loader = DataLoader(
        train_sub,
        batch_size=32,
        shuffle=True,
        num_workers=4
    )
    
    valid_loader = DataLoader(
        valid_sub,
        shuffle=False,
    )
    
    
    for epoch in range(num_epochs):
        train_loss = 0
        train_correct = 0
        train_total = 0
        
        model.train()
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            train_total += labels.size(0)
            train_correct += predicted.eq(labels).sum().item()
        
        # evaluate on validation set
        valid_correct = 0
        valid_total = 0
        
        model.eval()
        for images, labels in valid_loader:
            with torch.no_grad():
                images, labels = images.to(device), labels.to(device)
                output = model(images)
                _, predicted = output.max(1)
                valid_total += labels.size(0)
                valid_correct += predicted.eq(labels).sum().item()
        

        # update the learning rate
        if lr_scheduler is not None:
            lr_scheduler.step()

        print(f"[Fold {fold}] Epoch #{epoch} train_acc: {100.*train_correct/train_total}, valid_acc: {100.*valid_correct/valid_total}")   

[Fold 1] Epoch #0 train_acc: 84.82142857142857, valid_acc: 86.11111111111111
[Fold 1] Epoch #1 train_acc: 95.13227513227513, valid_acc: 85.7936507936508
[Fold 1] Epoch #2 train_acc: 96.42857142857143, valid_acc: 81.29629629629629
[Fold 1] Epoch #3 train_acc: 97.31481481481481, valid_acc: 82.46031746031746
[Fold 1] Epoch #4 train_acc: 97.3478835978836, valid_acc: 79.62962962962963
[Fold 1] Epoch #5 train_acc: 97.5, valid_acc: 79.81481481481481
[Fold 1] Epoch #6 train_acc: 97.91005291005291, valid_acc: 82.4074074074074
[Fold 1] Epoch #7 train_acc: 97.92328042328042, valid_acc: 83.46560846560847
[Fold 1] Epoch #8 train_acc: 97.90343915343915, valid_acc: 80.8994708994709
[Fold 1] Epoch #9 train_acc: 97.98280423280423, valid_acc: 61.34920634920635
[Fold 2] Epoch #0 train_acc: 96.0515873015873, valid_acc: 95.8994708994709
[Fold 2] Epoch #1 train_acc: 97.7579365079365, valid_acc: 94.92063492063492
[Fold 2] Epoch #2 train_acc: 98.11507936507937, valid_acc: 92.77777777777777
[Fold 2] Epoch #3 t

In [8]:
# Save model
PATH = 'model_9.pth'
torch.save(model.state_dict(), PATH)